In [266]:
import pandas as pd

df = pd.read_csv('../data/processed/headlines_cl_sent.csv')

In [267]:
from datetime import datetime
from datetime import timedelta
import time

In [268]:
df_site.columns

Index(['site', 'country_of_pub_x', 'bias', 'male_bias', 'empowerement',
       'polarity', 'violence', 'monthly_visits', 'country_of_pub_y'],
      dtype='object')

In [269]:
import ast
df['clean_hl_words'] = df['clean_hl_words'].apply(lambda row: ast.literal_eval(row))
# turn words into sets so we can compute intersection
df['clean_hl_words'] = df['clean_hl_words'].apply(lambda row: set([w.lower() for w in row]))
df['hl_words'] = df['headline_no_site'].apply(lambda row: set([w.lower() for w in row.split()]))

In [213]:
# def calc_publishing_time(scrape_date, duration):
#     date = 0
#     if duration[1] in ['week', 'weeks', 'weken']:
#         date = datetime(scrape_date) - timedelta(weeks=int(duration[0]))
#     elif duration[1] in ['days', 'day', 'dagen']:
#         date = datetime(scrape_date) - timedelta(days=int(duration[0]))
#     elif duration[1] in ['hour', 'hours', 'uur']:
#         date = datetime(scrape_date) - timedelta(hours=int(duration[0]))
#     return date
    

In [214]:
#calc_publishing_time(df['scrape_date'][0], df['time'].str.split(' ')[0])

In [270]:
#Two lists  of words that are used when a man or woman is present, based on Danielle Sucher's https://github.com/DanielleSucher/Jailbreak-the-Patriarchy
# male_words=set(['guy','spokesman','chairman',"men's",'men','him',"he's",'his','boy','boyfriend','boyfriends','boys','brother','brothers','dad','dads','dude','father','fathers','fiance','gentleman','gentlemen','god','grandfather','grandpa','grandson','groom','he','himself','husband','husbands','king','male','man','mr','nephew','nephews','priest','prince','son','sons','uncle','uncles','waiter','widower','widowers'])
female_words=set(['heroine','spokeswoman','chairwoman',"women's",'actress','women',"she's",'her','aunt',
                  'aunts','bride','daughter','daughters','female','fiancee','girl','girlfriend','girlfriends',
                  'girls','goddess','granddaughter','grandma','grandmother','herself','ladies','lady','lady',
                  'mom','moms','mother','mothers','mrs','ms','niece','nieces','priestess','princess',
                  'she','sister','sisters','waitress','widow','widows','wife','wives','woman', 'beautiful', 
                  'beauty', 'she', 'housewife', 'queen', 'gender', 'pregnant', 'queen', 'madam', 'mum', 'mummy', 
                  'mums', 'mistress'])

female_bias_words = set(['fair', 'weight', 'beauty', 'frail', 'weak', 'modest', 'virgin', 
                  'sex', 'feminine', 'sensitive', 'emotional', 'gentle', 'soft', 
                  'pretty', 'bitch', 'sexual', 'harassment', 'shopping','shop',
                  'sale', 'cook', 'cooking', 'sexual', 'baby', 'child', 'breast', 'breasts'
                 ,'affectionate', 'affection', 'cheer', 'emotion', 'emotions', 'feel', 'feeling'
                 , 'kind', 'kinship', 'modest', 'bossy', 'family', 'new born', 'newborn', 'new-born'
                 , 'nurture', 'nurtures', 'nurturing', 'pleasant', 'submissive', 'tender', 'cry', 'cries',
                         'crying', 'obedient'])

male_words = set(['hero', 'man',  'men', 'his', 'him', 'he', 'husband', 'father', 'male', 'son', 'god',
                  'prince', 'king', 'mr', 'sir', 'brother', 'grandfather', 'uncle', 'nephew', 'master', 'patriarch',
                 'chairman', 'chairmen'])

discrimination_words = set(['race', 'caste', 'casteless', 'black', 'SC', 'ST', 'african american', 'white', 'colour'])

male_bias_words = set([ "active", "adventurous",  "aggression",'aggressive', "ambition", "assert", 'assertive',
                       "athlete", 'athletic', "battle","champion", "decisive", 'head', 'dominate', 'dominant',
                        "driven", 'confident', 'strong', 'fight', 'force', 'master', 'superior', 'strength', 'bold', 
                       'ambitious', 'power', 'intelligent', 'greedy', 'hostile', 'uncaring', 'logic', 'logical', 'rational',
                       "fearless",'stubborn', 'independent', 'objective'])

empowerement_words = set(['chairperson', 'leader','leadership',  'chairwoman', 'minister', 'power','powerful', 'authority', 
                          'queen', 'manager', 'success', 'successful', 'successes', 'career', 'job',
                         'CEO','CFO', 'chief', 'officer', 'employment', 'employed', 'millionaire', 
                          'wealth', 'wealthy', 'strong', 'strength', 'courage'
                         , 'achievement', 'achievements', 'achieve', 'goal', 'ambition', 'ambitious',
                         'passionate','passion' ,'badass', 'confident', 'confidence', 'breakthrough',
                         'inspiring', 'inspiration', 'inspire', 'empower', 'empowered', 'empowerement'
                         'genius', 'expert', 'mastery', 'owner', 'businesswoman', 'intelligent', 'smart', 
                          'clever', 'wise', 'worth', 'role model', 'role-model', 'activist'])

violence_words = set(['violent', 'violence', 'crime', 'rape','rapist', 'raped', 'murder','kill', 'killed','killer',
         'murdered', 'murderer', 'attack', 'alleged', 'criminal', 'stab', 'knife', 'gun', 'guns', 'knives',
         'blood', 'bloodshed', 'court', 'rage', 'outrage', 'rob', 'steal', 'robber', 'stealer', 'beater',
         'domestic violence', 'aggression', 'aggressor', 'war', 'battle', 'abduction', 'assault', 'assaulted',
         'drug', 'abuse', 'child abuse', 'prison', 'fraud', 'human traffic', 'homicide', 'organised crime',
         'organised crime', 'genocide', 'fight', 'manslaughter', 'terrorist', 'weapon', 'smuggl', 'shoplift',
         'vandalism', 'crime', 'theft', 'penalty', 'prison sentence', 'detained', 'scold', 'guilty', 'trial',
         'defense', 'defend', 'cry', 'cries', 'crying', 'armed', 'jail', 'illegal', 'accomplice', 'accuse',
         'alcohol', 'allegation', 'arson', 'bail', 'battery', 'dead', 'death', 'deadly', 'corrupt', 'killer', 
         'sex crime', 'wanted'])

In [216]:
# def gender_the_sentence(sentence_words):
#     mw_length=len(male_words.intersection(sentence_words))
#     fw_length=len(female_words.intersection(sentence_words))

#     if mw_length>0 and fw_length==0:
#         gender='male'
#     elif mw_length==0 and fw_length>0: 
#         gender='female'
#     elif mw_length>0 and fw_length>0: 
#         gender='both'
#     else:
#         gender='none'
#     return gender

In [271]:
def gender_the_sentence(sentence_words, type_words):
#     mw_length=len(male_words.intersection(sentence_words))
    fw_length=len(type_words.intersection(sentence_words))

#     if mw_length>0 and fw_length==0:
#         gender='male'
    if fw_length>0: 
        
        gender=1
    else:
        gender=0
    return gender

In [272]:
def quantify_bias(sentence_words, type_words):
#     mw_length=len(male_words.intersection(sentence_words))
    type_length=len(type_words.intersection(sentence_words))
    bias_length = len(bias_words.intersection(sentence_words))
#     if bias_length>0:
#         print(sentence_words)
#         print(bias_length)

#     if mw_length>0 and fw_length==0:
#         gender='male'
    if type_length>0 and type_words == 'female_words': 
        bias=type_length + bias_length
    elif type_length>0 and type_words == 'male_words': 
        bias=type_length + len(male_bias_words.intersection(sentence_words))
    elif type_length>0 :
        bias = type_length
    else:
        bias=0
    return bias

In [273]:
df['clean_hl_words']

0        {oor, onderhoud, waarheid, moet, s, diana, wil...
1        {bekendes, doenig, die, ooievaar, k, plaaslike...
2                       {live, coast, dream, artist, west}
3                            {beckon, cillier, apple, big}
4        {bazier, title, city, alderman, p, councillor,...
                               ...                        
47549              {hospital, crash, life, fight, maghull}
47550    {slap, finger, treat, break, footballer, ex, c...
47551    {thug, blade, ear, bite, disabled, confront, b...
47552    {terminal, share, dad, heartbreaking, wish, yo...
47553    {mconald, wrong, worker, free, food, order, sh...
Name: clean_hl_words, Length: 47554, dtype: object

In [274]:
# create bias columns
df['gendered'] = df['hl_words'].apply(lambda row: gender_the_sentence(row, female_words))
df['bias'] = df['hl_words'].apply(lambda row: quantify_bias(row, female_words))
df['empowerement'] = df['hl_words'].apply(lambda row:quantify_bias(row, empowerement_words) )
#df['male_words'] = df['hl_words'].apply(lambda row:quantify_bias(row, male_words) )
df['male_bias'] = df['hl_words'].apply(lambda row:quantify_bias(row, male_words) )
df['violence'] = df['hl_words'].apply(lambda row:quantify_bias(row, violence_words) )

In [221]:
df[df['bias']>1].shape

(5213, 25)

In [275]:
df.columns

Index(['Unnamed: 0', 'url', 'headline', 'subtitle', 'time', 'scrape_date',
       'site', 'monthly_visits', 'country', 'country_of_pub',
       'headline_no_site', 'clean_headline', 'clean_subtitle',
       'clean_hl_words_no_women', 'clean_hl_words', 'compound_s', 'positive_s',
       'negative_s', 'neutral_s', 'hl_words', 'gendered', 'bias',
       'empowerement', 'male_bias', 'violence'],
      dtype='object')

In [276]:
import numpy as np
df['polarity'] = np.abs(df['compound_s'].values)

In [296]:
df_site = df.groupby(by=['site', 'country_of_pub']).agg({'bias': 'sum', 
                                      'empowerement': 'sum', 'monthly_visits' :'mean', 
                                                        'polarity' : 'mean',
                                                        'male_bias': 'sum', 'violence': 'sum'}).reset_index().sort_values(by = 'bias', ascending = False).reset_index(drop=True)

In [306]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
df_site['bias'] = Scaler.fit_transform(df_site[['bias']])

In [305]:
df_site['polarity'] = Scaler.fit_transform(df_site[['polarity']])

In [288]:
newsrooms = pd.read_csv('../data/raw/newsrooms.csv')
newsrooms.columns = ['site', 'monthly_visits', 'country', 'country_of_pub']
site_views = newsrooms.groupby(by='site').agg({'monthly_visits': 'sum'}).reset_index()
#df_site = site_views.join(df_site[['site', 'country_of_pub', 'bias', 'male_bias', 'empowerement', 'polarity', 'violence']].set_index('site'), on = 'site')

In [281]:
newsrooms.head()

,site,monthly_visits,country,country_of_pub
0,economictimes.indiatimes.com,0,India,India
1,aajtak.in,681400000,India,India
2,yahoo.com,85920000,India,USA
3,indiatimes.com,81500000,India,India
4,ndtv.com,77830000,India,India


In [300]:
df_site = pd.merge(df_site[['site', 'bias', 'male_bias', 'empowerement', 'polarity', 'violence', 'country_of_pub']], site_views[['site', 'monthly_visits']], on = 'site', how = 'inner')

In [301]:
df_site.shape

(186, 8)

In [307]:
df_site.to_csv('../data/processed/headlines_site.csv')

In [98]:
df_country

,country_of_pub,bias,male_words,empowerement,monthly_visits,site_count
0,USA,15140,1636,234,9.758486e+07,90
1,UK,9366,1094,144,6.463730e+07,43
2,India,6008,858,97,5.053054e+07,24
3,South Africa,3766,555,94,1.748276e+07,29
